In [10]:
import cost_model
import os
import json
import sys

from pprint import pprint
from IPython.display import SVG, display, HTML

def show_image(url1, url2):
    display(HTML("""<table><tr><td><img style='width:100px' src='{}'/></td>
                               <td><img style='width:100px' src='{}'/></td></tr></table>""".format(url1, url2)))

def load_design_v1(design_dir):
    """the directory containing exported designs, 
        the is the old version"""

    design = {}
    # load saved designs and trashed designs
    with open(os.path.join(design_dir, "saved.json")) as f:
        design["saved"] = json.load(f)["saved"]

    # load paths for saved svgs, relates to saved designs by their canvas ids
    svg_dir = os.path.join(design_dir, "saved_svgs")
    if not os.path.isdir(svg_dir):
        svg_dir = os.path.join(design_dir, "import_these_into_xd")

    for d in design["saved"]:
        d["img_id"] = d["id"]
    design["image_paths"] = {}
    for fname in os.listdir(svg_dir):
        if fname.endswith(".svg") or fname.endswith(".png"):
            design_id = fname.split(".")[0].split("-")[-1]
            design["image_paths"][design_id] = os.path.join(svg_dir, fname)

    design["dirname"] = design_dir
    return design

def load_design_v2(design_dir):
    """the directory containing exported designs, 
        the is the new version for user study data analysis"""
    design = {}
    # load saved designs and trashed designs
    with open(os.path.join(design_dir, "saved.json")) as f:
        design["saved"] = json.load(f)["saved"]

    # load paths for saved images, relates to saved designs by their canvas ids
    img_dir = os.path.join(design_dir, "images")

    for d in design["saved"]:
        d["img_id"] = d["elements"]["name"]
    design["image_paths"] = {}
    for fname in os.listdir(img_dir):
        if fname.endswith(".png"):
            if "Alternative 1" in fname:
                design_id = "Alternative 1"
            elif "Alternative 2" in fname:
                design_id = "Alternative 2"
            elif "Alternative 3" in fname:
                design_id = "Alternative 3"
            elif "Original" in fname:
                design_id = "Original"
                            
            design["image_paths"][design_id] = os.path.join(img_dir, fname)

    design["dirname"] = design_dir
    return design


def analyze_design(all_designs):
    """ Analyze design scores from all saved apps, the scores are normalized across all designs
        For each pair of saved designs in each file, display their diversity score as well as svgs
        Inputs:
            all_designs: a list of saved designs (saved.json)
    """
    
    raw_diversity_score = {}
    for exp_id, design in all_designs.items():          
        if len(design["saved"]) == 1:
            print("[error] this directory contains only 1 saved desigsn.")
            continue

        for i in range(len(design["saved"])):
            for j in range(i + 1, len(design["saved"])):
                di, dj = design["saved"][i], design["saved"][j]
                #diversity_score, diff = cost_model.compute_diversity_score(di["elements"], dj["elements"])
                raw_diversity_score[(exp_id, i, j)] = cost_model.compute_unnormalized_diversity_score(
                    di["elements"], dj["elements"])

    final_scores = cost_model.normalize_diversity_scores(raw_diversity_score)
    
    design_pairs = []
    for pair_id in final_scores:
        exp_id, i,j = pair_id
        score = (final_scores[pair_id]["alt_group_score"]
                    + final_scores[pair_id]["pos_diff_score"] 
                    + final_scores[pair_id]["rel_dist_diff_score"] 
                    + 2 * final_scores[pair_id]["size_diff_score"])
        design_pair = {
            "exp_id": exp_id,
            "score": score,
            "score_detail": final_scores[pair_id],
            "di": all_designs[exp_id]["saved"][i],
            "dj": all_designs[exp_id]["saved"][j],
        }
        
        design_pairs.append(design_pair)

    design_pairs.sort(key=lambda x: x['score'], reverse=True)
    for pair in design_pairs: 
        no_svg_to_display = False
        exp_id = pair["exp_id"]
        di = pair['di']
        dj = pair['dj']
        for d in [di, dj]:
            if d["img_id"] not in all_designs[exp_id]["image_paths"]:
                #print("  [Error] no svg for id {}".format(d["id"]))
                no_svg_to_display = True

        if no_svg_to_display:
            #print("  " + design["dirname"])
            continue
        score = pair['score']
        score_detail = pair['score_detail']
        print("# experiments from {}".format(pair["exp_id"]))
        print("# diversity score {}".format(score))
        print("# score details")
        pprint(score_detail)

        svg_i = all_designs[exp_id]["image_paths"][di["img_id"]]
        svg_j = all_designs[exp_id]["image_paths"][dj["img_id"]]
        show_image(svg_i, svg_j)
        print("")

def analyze_all_designs(export_dir, load_func):
    # iterate over all exported deisngs
    all_designs = {}
    for dirname in os.listdir(export_dir):
        design_dir = os.path.join(export_dir, dirname)
        if not os.path.isdir(design_dir):
            continue
        design = load_func(design_dir)
        all_designs[dirname] = design
    analyze_design(all_designs)

#analyze_all_designs("./saved_designs/old_designs", load_design_v1)
analyze_all_designs("./saved_designs/social_media", load_design_v2)

# experiments from 8_1_N_scout_social_media
# diversity score 1.5197064082539327
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.5541019831645061,
 'rel_dist_diff_score': 0.4275553673449126,
 'size_diff_score': 0.269024528872257}


,



# experiments from 2_2_P_baseline_social_media
# diversity score 1.4913118731470303
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.5552563939529869,
 'rel_dist_diff_score': 0.43369909124677825,
 'size_diff_score': 0.25117819397363267}


,



# experiments from 2_2_P_baseline_social_media
# diversity score 1.4849441045127902
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.7093628087161383,
 'rel_dist_diff_score': 0.4301628573952028,
 'size_diff_score': 0.1727092192007246}


...

,



# experiments from 20_2_P_baseline_social_media
# diversity score 0.23339454307715873
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.11238283312673636,
 'rel_dist_diff_score': 0.12101170995042239,
 'size_diff_score': 0.0}


,



# experiments from 15_2_P_baseline_social_media
# diversity score 0.20928640735821907
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.09992371561900995,
 'rel_dist_diff_score': 0.10936269173920912,
 'size_diff_score': 0.0}


,
